In [78]:
import os
import cv2
import numpy as np
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

# Diretório do conjunto de teste
TEST_DIR = "C:/Repositorios/Meus/PAI-Detecao-de-doencas-em-plantas-com-visao-computacional/data/raw/new-plant-diseases-dataset/test/test"

# Caminho do modelo salvo
MODEL_PATH = "C:/Repositorios/Meus/PAI-Detecao-de-doencas-em-plantas-com-visao-computacional/notebooks/modelos/PlantNet-DeepConv512-Robust_20241124_172312/plant_disease_model_20241124_174111.h5"

# Extraindo o nome do modelo automaticamente a partir do path
MODEL_NAME = os.path.basename(os.path.dirname(MODEL_PATH))

# Diretório para salvar os resultados
SAVE_DIR = os.path.join(
    r"C:/Repositorios/Meus/PAI-Detecao-de-doencas-em-plantas-com-visao-computacional/notebooks/testes",
    MODEL_NAME
)
os.makedirs(SAVE_DIR, exist_ok=True)

# Carregar o modelo treinado
model = load_model(MODEL_PATH)

In [79]:
# Verificar número de classes no modelo
num_classes_model = model.output_shape[-1]
print(f"Número de classes no modelo: {num_classes_model}")

# Listar as classes usadas no treinamento diretamente do diretório de treino
TRAIN_DIR = r"C:\Repositorios\Meus\PAI-Detecao-de-doencas-em-plantas-com-visao-computacional\data\raw\new-plant-diseases-dataset\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train"
# classes_selecionadas = [d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]
classes_selecionadas = [
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",
    # "Grape___Black_rot",
    # "Grape___Esca_(Black_Measles)",
    # "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    # "Grape___healthy",
    # "Orange___Haunglongbing_(Citrus_greening)",
    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Target_Spot",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___healthy"
]

# Verificar se o número de classes coincide
if len(classes_selecionadas) != num_classes_model:
    print(f"Alerta: O número de classes no modelo ({num_classes_model}) não corresponde ao número de classes treinadas ({len(classes_selecionadas)}).")


Número de classes no modelo: 15


In [80]:
# Função para carregar e preparar a imagem
def preparar_imagem(caminho_imagem, tamanho=(256, 256)):
    img = cv2.imread(caminho_imagem)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converte para RGB
    img = cv2.resize(img, tamanho)  # Redimensiona
    img = img_to_array(img) / 255.0  # Normaliza e converte para array
    return np.expand_dims(img, axis=0)  # Adiciona dimensão de batch

In [81]:
# Criar subpastas para resultados
pastas_resultados = {
    "Corretos": os.path.join(SAVE_DIR, "Corretos"),
    "Errados": os.path.join(SAVE_DIR, "Errados"),
    "Desconhecidos": os.path.join(SAVE_DIR, "Desconhecidos")
}
for pasta in pastas_resultados.values():
    os.makedirs(pasta, exist_ok=True)

# Processar imagens do conjunto de teste
for arquivo in os.listdir(TEST_DIR):
    caminho_completo = os.path.join(TEST_DIR, arquivo)
    if arquivo.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Preparar a imagem para predição
        img = preparar_imagem(caminho_completo)
        
        # Fazer a predição
        predicao = model.predict(img)
        if len(predicao[0]) != num_classes_model:
            print(f"Predição inesperada para a imagem {arquivo}: {predicao}")
            continue
        
        # Obter a classe prevista
        classe_prevista = classes_selecionadas[np.argmax(predicao)] if np.argmax(predicao) < len(classes_selecionadas) else "Desconhecido"
        
        # Extrair palavras do nome do arquivo separando por case (camelCase ou PascalCase)
        palavras_arquivo = [p.lower() for p in re.findall(r'[A-Z][a-z]*', arquivo.split('.')[0])]
        palavras_classe = classe_prevista.lower()
        
        # Verificar se todas as palavras do nome da imagem estão no nome da classe prevista
        correto = all(palavra in palavras_classe for palavra in palavras_arquivo)
        
        # Determinar a pasta de destino
        if classe_prevista == "Desconhecido":
            pasta_destino = pastas_resultados["Desconhecidos"]
        elif correto:
            pasta_destino = pastas_resultados["Corretos"]
        else:
            pasta_destino = pastas_resultados["Errados"]
        
        # Visualizar resultado
        img_original = cv2.imread(caminho_completo)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)  # Converte para RGB
        
        plt.figure(figsize=(8, 6))
        plt.imshow(img_original)
        plt.axis("off")
        plt.title(
            f"Imagem Atual: {arquivo}\n"
            f"Previsto: {classe_prevista}\n"
            f"{'Correto' if correto else 'Errado'}",
            fontsize=10, color='green' if correto else 'red'
        )
        
        # Salvar visualização na pasta apropriada
        plt.savefig(os.path.join(pasta_destino, f"{arquivo}_resultado.png"), dpi=300, bbox_inches='tight')
        plt.close()

print(f"Resultados visuais salvos nas pastas: {list(pastas_resultados.values())}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━